In [ ]:
# Environment variables
drive = False
install_packages = False

# If necessary, install packages
if install_packages:
  !pip install Basemap basemap-data-hires
  !pip install pandas matplotlib numpy
  !pip install sklearn
  !pip install datetime

# Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from csv import reader
from sklearn.metrics import r2_score
from datetime import datetime
from datetime import timedelta as tdelta
from datetime import time as time
import warnings
import osmnx as ox
import folium
import networkx as nx
from matplotlib import colors

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Mount drive if needed
if drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/LSIS

from Helper_funcs import *
from SensorPositions_Func import *
from SVD_Func import *

# Depending on the environemnt, set directory to data
if drive:
  dir_path = '/content/drive/MyDrive/LSIS/data/'
else :
  dir_path = './data/'

folders = ['region_1_mustamäe_kristiine', 'region_2_data_kesklinn', 'region_3_kadriorg_lasnamäe', 'region_4_ülemiste']
columns_sensors_positions = ['IDs', 'latitude', 'longitude', 'region']

# 1) Import Data #

In [ ]:
csv_file_name = 'sensor_positions.csv'
coords = []
IDs = []

with open(dir_path + csv_file_name, 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        coord = (float(row[0].split(' ')[0].replace('(', '')), 
                 float(row[0].split(' ')[1].replace(')', '')) )
        coords.append(coord)
        
        IDs.append(row[1])

stations_df = import_sensor_positions(dir_path, csv_file_name)
stations_df.set_index('IDs', inplace=True)
IDs = list(stations_df.index)

del csv_file_name, coords, csv_reader, read_obj, row

In [ ]:
df_data_interp = pd.read_csv(dir_path + 'interpolated_data_1.csv');
tindex = pd.to_datetime(df_data_interp[df_data_interp.columns[1]]);
df_data_interp.set_index(tindex, inplace=True);
df_data_interp.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis = 1, inplace=True);

df_day = df_data_interp.iloc[indexer_day(df_data_interp)]
df_eve = df_data_interp.iloc[indexer_evening(df_data_interp)]
df_night = df_data_interp.iloc[indexer_night(df_data_interp)]

stations_df['m_day'] = df_day.mean()
stations_df['m_eve'] = df_eve.mean()
stations_df['m_nig'] = df_night.mean()



del df_data_interp, df_day, df_eve, df_night


# 2) Download maps #

In [ ]:


lower_left = (59 + 23/60 + 9.8/3600, 24 + 38/60 + 13.1/3600)
upper_right = (59 + 26/60 + 35.8/3600, 24 + 47/60 + 23.4/3600)

graph = ox.graph_from_bbox(upper_right[0], lower_left[0], upper_right[1], lower_left[1], network_type='drive')
data = ox.graph_to_gdfs(graph, nodes=False)

# Calculate the center of the map
center = ((lower_left[0] + upper_right[0]) / 2, (lower_left[1] + upper_right[1]) / 2)

# Create a folium map centered around the calculated center


# 3) Assign sensors to streets + erase redundant #

In [ ]:
tdata = [[] for _ in range(len(stations_df.index))]
stations_df['nearest_ed']= tdata
stations_df['nearest_ed1']= tdata
nearest_edges = []

for sid in stations_df.index:
    stations_df['nearest_ed'][sid] = ox.distance.nearest_edges(graph, X=stations_df['latitude'][sid], Y=stations_df['longitude'][sid])
    
    nearest_edges.append(stations_df['nearest_ed'][sid])

graph_temp = graph.copy()

for sid in stations_df.index:
    graph_temp.remove_edge(stations_df['nearest_ed'][sid][0], stations_df['nearest_ed'][sid][1])
    stations_df['nearest_ed1'][sid] = ox.distance.nearest_edges(graph, X=stations_df['latitude'][sid], Y=stations_df['longitude'][sid])
    graph_temp = graph.copy()
    nearest_edges.append(stations_df['nearest_ed1'][sid])
    
nearest_edges = pd.DataFrame(data={'edges':nearest_edges})
nearest_edges.drop_duplicates(subset='edges', keep='first', inplace=True)
    


del tdata, graph_temp

In [ ]:
graph3 = graph.edge_subgraph(nearest_edges['edges'])
nearest_edges.set_index(nearest_edges['edges'], inplace=True)

# 4) Assign colors #

In [17]:
# Choosing a colormap
colormap = plt.cm.get_cmap('jet')  # You can change 'cool' to any other available colormap

# Normalizing the values between 0 and 1
norm = plt.Normalize(30, 70)


nearest_edges['ddB1'] = np.zeros(len(nearest_edges))
nearest_edges['ddB2'] = np.zeros(len(nearest_edges))
nearest_edges['ddB'] = np.zeros(len(nearest_edges))
nearest_edges['edB1'] = np.zeros(len(nearest_edges))
nearest_edges['edB2'] = np.zeros(len(nearest_edges))
nearest_edges['edB'] = np.zeros(len(nearest_edges))
nearest_edges['ndB1'] = np.zeros(len(nearest_edges))
nearest_edges['ndB2'] = np.zeros(len(nearest_edges))
nearest_edges['ndB'] = np.zeros(len(nearest_edges))


for edge in nearest_edges.index:
    idxs = stations_df['nearest_ed'] == edge;
    idxs1 = stations_df['nearest_ed1'] == edge;
    nearest_edges['ddB1'][edge] = stations_df['m_day'][idxs].mean()
    nearest_edges['edB1'][edge] = stations_df['m_eve'][idxs].mean()
    nearest_edges['ndB1'][edge] = stations_df['m_nig'][idxs].mean()
    nearest_edges['ddB2'][edge] = stations_df['m_day'][idxs1].mean()
    nearest_edges['edB2'][edge] = stations_df['m_eve'][idxs1].mean()
    nearest_edges['ndB2'][edge] = stations_df['m_nig'][idxs1].mean()
    
nearest_edges['ddB'] = .5 * nearest_edges['ddB1'] + nearest_edges['ddB2'];
nearest_edges['edB'] = .5 * nearest_edges['edB1'] + nearest_edges['edB2'];
nearest_edges['ndB'] = .5 * nearest_edges['ndB1'] + nearest_edges['ndB2'];

nearest_edges.drop(labels=['ddB1', 'ddB2', 'edB1', 'edB2', 'ndB1', 'ndB2'], inplace=True, axis=1)

# Creating a list of colors corresponding to the values
nearest_edges['day'] = [colormap(norm(val)) for val in nearest_edges['ddB']]
nearest_edges['eve'] = [colormap(norm(val)) for val in nearest_edges['edB']]
nearest_edges['nig'] = [colormap(norm(val)) for val in nearest_edges['ndB']]

nearest_edges['hday'] = to_hex()


C:\Users\radek\AppData\Local\Temp\ipykernel_6556\2631887194.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = plt.cm.get_cmap('jet')  # You can change 'cool' to any other available colormap


KeyError: "['ddB1', 'ddB2', 'edB1', 'edB2', 'ndB1', 'ndB2'] not found in axis"

# X) Print map #

In [ ]:
import ast
m = folium.Map(location=center, zoom_start=13)

data = ox.graph_to_gdfs(graph3, nodes=False)

for (lati, long) in zip(stations_df['latitude'], stations_df['longitude']):
    folium.Marker(location=(long, lati)).add_to(m)

# Define a function to style the road features
def style_function(feature):
    id = ast.literal_eval(feature.get('id'));
    color = 'red';
    if len(id) == 3:
        color = nearest_edges['heve'][id];
    return {
        'color': color,  # Set the color of the roads (red in this example)
        'weight': 8,         # Set the line weight
        'opacity': 0.7       # Set the line opacity
    }

# Add the road network to the folium map as a GeoJSON layer with the specified style
folium.GeoJson(data, style_function=style_function).add_to(m)
m


In [ ]:
for edge in nearest_edges.index:
    graph3.edges[edge]['color'] = '#beff39'
    
for node in graph3.nodes:
    graph3.nodes[node]['color'] = '#beff39'

In [19]:
nearest_edges.drop(labels=['ddB1', 'ddB2', 'edB1', 'edB2', 'ndB1', 'ndB2'], inplace=True, axis=1)